In [70]:
#imports
import os
import pandas as pd
import networkx as nx
from matplotlib.pyplot import figure
import numpy as np
import json
from openpyxl import load_workbook


In [ ]:
#Lista os arquivos da pasta
def getFileName(filepath):
    files = os.listdir(filepath)
    return files

In [75]:
#Lista as sheets dos arquivos
def getSheetnamesXlsx(filepath):
    wb = load_workbook(filepath, read_only=True, keep_links=False)
    return wb.sheetnames

['PORTAS_CORE']

In [ ]:
getSheetnamesXlsx("planilhas/cmn01.xlsx")


In [ ]:
#converte excel para Json
excel_data_df = pd.read_excel('planilhas/pacs01.xlsx', sheet_name='PORTAS_CORE')

json_str = excel_data_df.to_json(orient='records')
print(json_str)

In [68]:
#estrutura do json
parsed = json.loads(json_str)
print(json.dumps(parsed, indent=4, sort_keys=False))


[
    {
        "Porta": 1,
        "Conex\u00e3o": "CORE01",
        "Descri\u00e7\u00e3o": "ISL PORT",
        "DIO": "PATCH CORD",
        "Velocidade ( Giga )": 10,
        "Altera\u00e7\u00e3o": null
    },
    {
        "Porta": 2,
        "Conex\u00e3o": "AGG_PACS",
        "Descri\u00e7\u00e3o": "PO 101",
        "DIO": "TWINAX",
        "Velocidade ( Giga )": 10,
        "Altera\u00e7\u00e3o": null
    },
    {
        "Porta": 17,
        "Conex\u00e3o": "INRAD01",
        "Descri\u00e7\u00e3o": "ISL PORT",
        "DIO": "RX2/TX2 ( Jumper NETI + Anel )",
        "Velocidade ( Giga )": 10,
        "Altera\u00e7\u00e3o": null
    },
    {
        "Porta": 18,
        "Conex\u00e3o": "AGG_PACS",
        "Descri\u00e7\u00e3o": "PO 101",
        "DIO": "TWINAX",
        "Velocidade ( Giga )": 10,
        "Altera\u00e7\u00e3o": null
    },
    {
        "Porta": 19,
        "Conex\u00e3o": "CORE02",
        "Descri\u00e7\u00e3o": "ISL PORT",
        "DIO": "PATCH CORD",
        "V

In [ ]:
#Lidando com NaN e o index
df.fillna(0, inplace=True)
df = df.set_index("switch")
df = df.astype('int')
df

In [ ]:
# df['S0_regex'] = df['S0'].str.extract(r"\([[0-9][0-9]*)\]", expand=False)
#verifica conexão dos nós
df1 = df != 0
df1

In [ ]:
#colunas: fonte e alvos
source = df1.iloc[:,1:].columns
targets = df1.iloc[:,1:].columns

In [ ]:
source

In [ ]:
targets

In [ ]:
#populando os nós e arestas
G =nx.from_pandas_adjacency(df, create_using = nx.Graph())
pos=nx.shell_layout(G)# positions for all nodes

In [ ]:
print(nx.info(G))

In [ ]:
print("Grau de centralidade")
d = nx.degree_centrality(G)
for v in G.nodes():
    print(f"{v:2} {d[v]:.3f}")

In [ ]:
#grafo
figure(figsize=(12,12))
nx.draw_shell(G, with_labels=True, node_size=6000)
nx.draw_networkx_edge_labels(G,pos=pos)

